In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision
from torch.autograd import Variable
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
import pickle
from PIL import Image
from tensorboardX import SummaryWriter

In [2]:
# for CRNN
class Dataset_CRNN(data.Dataset):
    "Characterizes a dataset for PyTorch"
    def __init__(self, data_path, frame_length=10, transform=None):
        "Initialization"
        self.transform = transform
        #self.frames = frames
        self.folders = data_path
        self.frames = frame_length #For our case since we are computing 10 frames always

    def __len__(self):
        "Denotes the total number of samples"
        return len(os.listdir(self.folders))

    def read_images(self, data_path, use_transform):
        X = []
        file_name = ""
        for i in os.listdir(data_path):
            file_name = i
            image = Image.open(os.path.join(data_path,i))
            
            #print(image.shape)
            if use_transform is not None:
                image = use_transform(image)
                #print(image.size)
            image = torch.from_numpy(np.asarray(image))
            X.append(image)
        X = torch.stack(X, dim=0)

        return X, file_name

    def __getitem__(self, index):
        data_path = os.path.join(self.folders,os.listdir(self.folders)[index])
              
        # Load data
        X, file_name = self.read_images(data_path, self.transform)                     # (input) spatial images
        
        y = np.ones(self.frames)
        if 'real' in data_path:
            y = np.zeros(self.frames)
        #print("Folder is {}".format(data_path))
        #print(X.shape)
        return X, torch.from_numpy(y).type(torch.LongTensor), file_name

In [3]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(256),
    #transforms.ToTensor()
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         #std=[0.229, 0.224, 0.225] )
    ])

In [4]:
train_path = '/home/chinmay/datatset/deepfake_split/train'
train_data = Dataset_CRNN(train_path, transform=TRANSFORM_IMG, frame_length=30 )
# for step, (x, y) in enumerate(data):
#     print(x.shape)
val_path = '/home/chinmay/datatset/deepfake_split/val'
val_data = Dataset_CRNN(val_path, transform=TRANSFORM_IMG, frame_length=30)

In [5]:
batch_size = 8
epochs = 40
log_interval = 20
learning_rate = 1e-4

In [6]:
from Meso4 import Meso4

ImportError: No module named 'Meso4'

In [7]:
# Detect devices
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu") 
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
train_loader = data.DataLoader(train_data, **params)
valid_loader = data.DataLoader(val_data, **params)
#model = Meso4()
#model.cuda()
# for index, (X,y) in enumerate(train_loader):
#     y_pred = model(X)
#     print (y_pred.shape)
# Shape was torch.Size([4, 10, 2])
model = Meso4()
model.cuda()

Meso4(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm_1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (conv2): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool_2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (max_pool_4): MaxPool2d(kernel_size=(4, 4), stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc_conv): Conv2d(16, 2, kernel_size=(16, 16), stride=(1, 1))
  (dropout): Dropout2d(p=0.

In [8]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
def find_median(numpy_array = []): #This is expected to take an array of array. So,
    #print("Input array is {}".format(numpy_array))
    output = []
    for array in numpy_array:
        counts = np.bincount(array)
        output.append(np.argmax(counts))
    return torch.from_numpy(np.asarray(output)).type(torch.LongTensor)

In [10]:
writer_train = SummaryWriter('/home/chinmay/training-results/df-dropout/train')
writer_test = SummaryWriter('/home/chinmay/training-results/df-dropout/test')
save_model_path = "/home/chinmay/model_weights/MesoNet-df/MesoNet-dropout-40"
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train() #Put the model in training mode
    losses = []
    N_count = 0   # counting total trained sample in one epoch
    scores = []
    #single_iter_loss = []
    for batch_idx, (X, y, file_name) in enumerate(train_loader):
        # distribute data to device
        X, y = X.to(device), y.to(device)
        N_count += X.size(0)

        optimizer.zero_grad()
        output = model(X)   # output has dim = (batch, number of classes)
        #print(output)
        #print(y.shape)
        loss_train = 0
        for items in range(output.shape[1]):
            loss = loss_function(output[:,items,:], y[:,items])
            loss_train = loss_train + loss.item()
            #single_iter_loss.append(loss)
            loss.backward(retain_graph=True)
            optimizer.step()
        #print("backward update")
        #loss_batch = np.sum(single_iter_loss)
        #loss_batch.backward(retain_graph=True) #Possible improvement to increase speed
        #optimizer.step()
        
        losses.append(loss_train)

        # to compute accuracy
        y_pred = torch.max(output, 2)[1]  # y_pred != output
        # We take the modal value of all predictions. So,
        y = find_median(y)
        y_pred = find_median(y_pred)
        #print("filename is {}".format(file_name))
        #print("prediction is {}".format(y_pred))
        
        step_score = accuracy_score(y.cpu().data.squeeze().numpy(), y_pred.cpu().data.squeeze().numpy())
        scores.append(step_score)         # computed on CPU
        #print(step_score)
        

        # show information
                 
        if (batch_idx + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, N_count, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss_train, 100 * step_score))
        
    # we should sum all these losses and then divide by the training batch size
    losses_scalar = np.sum(losses)/ len(train_loader.dataset) 
    return losses_scalar, np.mean(scores)

In [11]:
def validation(model, device, optimizer, test_loader):
    # set model as testing mode
    model.eval()

    test_loss = 0
    all_y = []
    all_y_pred = []
    with torch.no_grad():
        for X, y,file_name in test_loader:
            # distribute data to device
            X, y = X.to(device), y.to(device)

            output = model(X)
            #print("Output shape is {}".format(output.shape))
            #print("Expected shape is {}".format(y.shape))
            for items in range(output.shape[1]):
                loss = loss_function(output[:,items,:], y[:,items])
                test_loss += loss.item()                 # sum up batch loss
            #Predict value
            y_pred = torch.max(output, 2)[1]  # (y_pred != output) get the index of the max log-probability
            # Replace the value with the median value
            y_pred = find_median(y_pred)
            y = find_median(y)
            print("actual value of labels read from disc is {}".format(y))
            print("filename is {} and prediction is {}".format(file_name, y_pred))
            # collect all y and y_pred in all batches
            all_y.extend(y)
            all_y_pred.extend(y_pred)

    test_loss /= len(test_loader.dataset)

    # compute accuracy
    all_y = torch.stack(all_y, dim=0)
    all_y_pred = torch.stack(all_y_pred, dim=0)
    test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())

    # show information
    print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_y), test_loss, 100* test_score))

    # save Pytorch models of best record
    torch.save(model.state_dict(), os.path.join(save_model_path, 'meso_df_2_epoch{}.pth'.format(epoch + 1)))  # save spatial_encoder
    torch.save(optimizer.state_dict(), os.path.join(save_model_path, 'meso_df_2_optimizer_epoch{}.pth'.format(epoch + 1)))      # save optimizer
    

    return test_loss, test_score

In [12]:
def adjust_learning_rate(optimizer, learning_rate, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 10 epochs"""
    lr = learning_rate * (0.1 ** (epoch // 10))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [13]:
epoch_train_losses = []
epoch_train_scores = []
epoch_test_losses = []
epoch_test_scores = []
for epoch in range(epochs):
    # train, test model
    train_losses, train_scores = train(log_interval, model, device, train_loader, optimizer, epoch)
    epoch_test_loss, epoch_test_score = validation(model, device, optimizer, valid_loader)
    # Reduce learning-rate by a factor of 1/10 after every 10 epochs
    adjust_learning_rate(optimizer=optimizer, learning_rate=learning_rate, epoch=epoch)
    # save results
    writer_train.add_scalar('loss',train_losses,epoch+1)
    writer_train.add_scalar('score',train_scores,epoch+1)
    writer_test.add_scalar('loss',epoch_test_loss,epoch+1)
    writer_test.add_scalar('score',epoch_test_score,epoch+1)
    epoch_train_losses.append(train_losses)
    epoch_train_scores.append(train_scores)
    epoch_test_losses.append(epoch_test_loss)
    epoch_test_scores.append(epoch_test_score)
    #Empty the cache
    torch.cuda.empty_cache()


Train Epoch: 1 [160/416 (38%)]	Loss: 29.054499, Accu: 75.00%
Train Epoch: 1 [320/416 (77%)]	Loss: 24.140595, Accu: 50.00%
actual value of labels read from disc is tensor([0, 1, 0, 1, 1, 0, 1, 1])
filename is ['54_228.jpg', '170_1378.jpg', '71_214.jpg', '145_280.jpg', '148_0.jpg', '63_276.jpg', '168_1250.jpg', '115_6.jpg'] and prediction is tensor([0, 1, 0, 0, 0, 0, 0, 0])
actual value of labels read from disc is tensor([0, 1, 0, 0, 0, 0, 0, 0])
filename is ['81_182.jpg', '152_316.jpg', '82_381.jpg', '79_459.jpg', '55_126.jpg', '63_507.jpg', '68_148.jpg', '90_895.jpg'] and prediction is tensor([0, 0, 0, 0, 0, 0, 0, 0])
actual value of labels read from disc is tensor([1, 0, 0, 0, 0, 1, 0, 0])
filename is ['116_896.jpg', '48_240.jpg', '102_254.jpg', '98_178.jpg', '86_316.jpg', '173_67.jpg', '44_0.jpg', '87_620.jpg'] and prediction is tensor([0, 0, 0, 0, 0, 0, 0, 0])
actual value of labels read from disc is tensor([0, 0, 0, 0, 0, 0, 0, 0])
filename is ['82_228.jpg', '88_253.jpg', '82_96.jp

actual value of labels read from disc is tensor([0, 1, 0, 0, 0, 0, 0, 1])
filename is ['82_96.jpg', '167_212.jpg', '35_84.jpg', '70_342.jpg', '82_381.jpg', '72_196.jpg', '87_620.jpg', '176_48.jpg'] and prediction is tensor([0, 1, 0, 0, 1, 0, 0, 0])
actual value of labels read from disc is tensor([0, 1, 0, 0, 0, 1, 1, 0])
filename is ['64_148.jpg', '118_309.jpg', '76_138.jpg', '70_234.jpg', '45_168.jpg', '170_1378.jpg', '142_176.jpg', '52_270.jpg'] and prediction is tensor([0, 1, 0, 0, 0, 1, 1, 0])
actual value of labels read from disc is tensor([0, 1, 0, 1, 1, 1, 1, 1])
filename is ['39_66.jpg', '168_110.jpg', '87_900.jpg', '165_393.jpg', '166_89.jpg', '154_50.jpg', '134_405.jpg', '123_92.jpg'] and prediction is tensor([0, 1, 0, 1, 1, 1, 1, 1])
actual value of labels read from disc is tensor([1, 1, 1, 1, 1, 0, 0, 0])
filename is ['125_2.jpg', '161_172.jpg', '127_412.jpg', '170_234.jpg', '162_86.jpg', '83_57.jpg', '48_240.jpg', '62_612.jpg'] and prediction is tensor([1, 1, 1, 1, 1, 0, 0

actual value of labels read from disc is tensor([1, 1, 1, 0, 1, 1, 0, 1])
filename is ['118_171.jpg', '142_176.jpg', '155_260.jpg', '102_42.jpg', '128_1530.jpg', '176_150.jpg', '72_130.jpg', '124_186.jpg'] and prediction is tensor([1, 1, 1, 1, 1, 1, 1, 1])
actual value of labels read from disc is tensor([0, 0, 0, 0, 0, 0, 0, 1])
filename is ['85_242.jpg', '97_62.jpg', '99_298.jpg', '86_150.jpg', '83_393.jpg', '100_4.jpg', '73_436.jpg', '119_580.jpg'] and prediction is tensor([1, 1, 0, 1, 0, 1, 1, 1])
actual value of labels read from disc is tensor([0, 0, 1, 0, 0, 0, 0, 0])
filename is ['81_182.jpg', '94_267.jpg', '176_48.jpg', '82_228.jpg', '38_288.jpg', '74_188.jpg', '77_447.jpg', '102_254.jpg'] and prediction is tensor([1, 1, 1, 1, 1, 1, 0, 0])
actual value of labels read from disc is tensor([0, 0, 0, 1, 0, 0, 1, 1])
filename is ['45_48.jpg', '56_84.jpg', '77_534.jpg', '116_896.jpg', '45_168.jpg', '87_900.jpg', '146_252.jpg', '152_280.jpg'] and prediction is tensor([1, 1, 0, 1, 0, 1,

actual value of labels read from disc is tensor([0, 0, 1, 1, 0, 1, 0, 0])
filename is ['101_82.jpg', '68_148.jpg', '168_1250.jpg', '124_186.jpg', '68_720.jpg', '152_280.jpg', '69_196.jpg', '71_344.jpg'] and prediction is tensor([0, 0, 1, 1, 1, 1, 0, 0])
actual value of labels read from disc is tensor([1, 1, 0, 0, 0, 1, 0, 1])
filename is ['144_108.jpg', '149_117.jpg', '94_267.jpg', '48_12.jpg', '73_132.jpg', '131_92.jpg', '81_68.jpg', '150_26.jpg'] and prediction is tensor([0, 1, 0, 0, 0, 1, 0, 1])
actual value of labels read from disc is tensor([1, 1, 0, 1, 0, 0, 1, 1])
filename is ['132_93.jpg', '164_112.jpg', '50_96.jpg', '144_66.jpg', '85_242.jpg', '91_252.jpg', '147_108.jpg', '170_1378.jpg'] and prediction is tensor([1, 1, 0, 1, 1, 0, 1, 1])
actual value of labels read from disc is tensor([1, 0, 1, 0, 1, 0, 0, 0])
filename is ['135_800.jpg', '68_664.jpg', '165_33.jpg', '102_42.jpg', '117_12.jpg', '44_72.jpg', '70_342.jpg', '100_4.jpg'] and prediction is tensor([1, 1, 1, 0, 1, 0, 1

actual value of labels read from disc is tensor([0, 0, 1, 0, 0, 1, 0, 1])
filename is ['58_138.jpg', '65_82.jpg', '169_864.jpg', '40_12.jpg', '81_68.jpg', '128_1122.jpg', '82_96.jpg', '170_234.jpg'] and prediction is tensor([0, 0, 1, 0, 0, 1, 0, 1])
actual value of labels read from disc is tensor([0, 1, 0, 0, 1, 1, 0, 1])
filename is ['82_228.jpg', '150_26.jpg', '88_161.jpg', '68_720.jpg', '128_1530.jpg', '172_600.jpg', '88_40.jpg', '168_1250.jpg'] and prediction is tensor([0, 1, 0, 0, 1, 1, 0, 1])
actual value of labels read from disc is tensor([1, 1, 1, 0, 1, 0, 1, 0])
filename is ['144_108.jpg', '135_1200.jpg', '131_92.jpg', '35_84.jpg', '120_725.jpg', '74_816.jpg', '126_300.jpg', '67_534.jpg'] and prediction is tensor([1, 1, 1, 0, 1, 0, 1, 0])
actual value of labels read from disc is tensor([0, 1, 0, 0, 1, 1, 1, 1])
filename is ['102_254.jpg', '144_66.jpg', '90_895.jpg', '53_360.jpg', '151_280.jpg', '155_425.jpg', '133_327.jpg', '134_405.jpg'] and prediction is tensor([0, 1, 0, 0, 

...

actual value of labels read from disc is tensor([0, 1, 0, 0, 0, 1, 1, 0])
filename is ['37_150.jpg', '162_86.jpg', '67_534.jpg', '76_138.jpg', '92_1056.jpg', '157_594.jpg', '132_228.jpg', '101_82.jpg'] and prediction is tensor([0, 1, 0, 0, 0, 1, 1, 0])
actual value of labels read from disc is tensor([0, 0, 0, 0, 1, 0, 0, 0])
filename is ['64_438.jpg', '83_57.jpg', '44_0.jpg', '100_824.jpg', '114_396.jpg', '73_436.jpg', '35_84.jpg', '38_288.jpg'] and prediction is tensor([0, 0, 0, 0, 1, 0, 0, 0])
actual value of labels read from disc is tensor([0, 0, 0, 0, 0, 0, 1, 1])
filename is ['82_381.jpg', '91_252.jpg', '74_816.jpg', '83_393.jpg', '67_354.jpg', '42_180.jpg', '150_26.jpg', '126_300.jpg'] and prediction is tensor([0, 0, 0, 0, 0, 0, 1, 1])
actual value of labels read from disc is tensor([0, 0, 1, 0, 1, 0, 1, 1])
filename is ['88_40.jpg', '60_426.jpg', '143_39.jpg', '62_30.jpg', '155_425.jpg', '75_513.jpg', '132_93.jpg', '118_309.jpg'] and prediction is tensor([0, 0, 1, 0, 1, 0, 1, 1]

actual value of labels read from disc is tensor([1, 0, 0, 0, 0, 1, 0, 1])
filename is ['130_7.jpg', '54_228.jpg', '64_438.jpg', '51_72.jpg', '84_273.jpg', '157_171.jpg', '65_132.jpg', '168_110.jpg'] and prediction is tensor([1, 0, 0, 0, 0, 1, 0, 1])
actual value of labels read from disc is tensor([0, 0, 1, 1, 1, 0, 0, 0])
filename is ['49_1410.jpg', '47_246.jpg', '123_106.jpg', '126_565.jpg', '125_110.jpg', '51_174.jpg', '99_140.jpg', '50_96.jpg'] and prediction is tensor([0, 0, 1, 1, 1, 0, 0, 0])
actual value of labels read from disc is tensor([1, 0, 1, 1, 0, 1, 0, 0])
filename is ['146_252.jpg', '89_670.jpg', '135_1200.jpg', '144_66.jpg', '64_148.jpg', '143_39.jpg', '96_320.jpg', '61_1290.jpg'] and prediction is tensor([1, 0, 1, 1, 0, 1, 0, 0])
actual value of labels read from disc is tensor([0, 0, 1])
filename is ['48_12.jpg', '82_96.jpg', '158_33.jpg'] and prediction is tensor([0, 0, 1])

Test set (235 samples): Average loss: 0.9813, Accuracy: 95.74%

Train Epoch: 33 [160/416 (38%)

Train Epoch: 34 [160/416 (38%)]	Loss: 2.868134, Accu: 100.00%
Train Epoch: 34 [320/416 (77%)]	Loss: 8.202102, Accu: 100.00%
actual value of labels read from disc is tensor([0, 0, 0, 1, 1, 0, 0, 1])
filename is ['44_72.jpg', '89_490.jpg', '98_178.jpg', '126_565.jpg', '116_296.jpg', '77_447.jpg', '48_12.jpg', '165_393.jpg'] and prediction is tensor([0, 0, 0, 1, 1, 0, 0, 1])
actual value of labels read from disc is tensor([1, 0, 0, 0, 0, 0, 1, 1])
filename is ['158_33.jpg', '74_816.jpg', '40_12.jpg', '53_360.jpg', '79_93.jpg', '78_828.jpg', '167_212.jpg', '168_90.jpg'] and prediction is tensor([1, 0, 0, 0, 0, 0, 1, 1])
actual value of labels read from disc is tensor([0, 0, 0, 1, 1, 0, 0, 0])
filename is ['70_342.jpg', '85_242.jpg', '88_40.jpg', '117_12.jpg', '142_120.jpg', '60_504.jpg', '65_82.jpg', '45_168.jpg'] and prediction is tensor([0, 0, 0, 1, 1, 0, 0, 0])
actual value of labels read from disc is tensor([1, 1, 0, 0, 1, 1, 0, 0])
filename is ['175_68.jpg', '146_438.jpg', '86_150.jpg

Process Process-276:
Process Process-275:
Process Process-273:
Process Process-274:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/chinmay/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/chinmay/anaconda3/lib/python3.5/

KeyboardInterrupt: 

In [14]:
torch.cuda.empty_cache()

In [ ]:
# E